# Study Case 02 - Data Mining dan Business Intelligence : Customer Segmentation untuk MixITup Ice Cream

**Kelompok 3 :**
1. Michael Mario Bramanthyo Adhi    (1906299534) (Ketua)
2. Amanda Sarah Ivana               (1906375814)
3. Divina Fatriandira               (1906299465)
4. Eslim Suyangsu                   (1906299471)
5. Hunaiva Kintan                   (1906375695)
6. Karin Marshanda                  (1906299515)
7. Rosyda Hanavania                 (1906299566)
8. Zalfa Nurfadhilah Haris          (1906305291)

## Kasus: MixITUp Ice Cream
- **MixITup** adalah perusahaan rintisan yang dimotori para anak muda yang bergerak di bidang usaha Ice Cream, Boba Drinks, dan Desserts (snacks) \
  Saat ini MixITup memiliki 3 cabang di Jakarta, Depok, dan Tangerang. MixITup mendapat respon positif di masyarakat, antrian dine-in dan pesan-antar produk MixITup melebihi kapasitas pelayanan. \
  Bulan lalu MixITup mendapatkan pendanaan seri B sebesar Rp. 33 Milyar,- \
  Rencananya MixITup ingin "scale" business-nya ke kota lain dan melakukan massive marketing. 

- Melalui iming-iming diskon 5% untuk beberapa item di tokonya dan pembagian voucher promo diskon, MixITup berhasil mengumpulkan data pelanggan melalui kartu membernya (loyality program). \
  Anda adalah Data Analyst junior di MixITup, tugas anda adalah melakukan proses Data Mining dan memberikan informasi/insight dan saran/rekomendasi/strategi ke perusahaan via data.

## Petunjuk
*Catatan:*	
0-	MixITup belum memiliki Data Engineer, file yang diberikan oleh perusahaan akan bertipe/format "Excel"!!!...
	Warning … Karena file berbentuk Excel anda harus menyiapkan mekanisme agar file tersebut dapat di load secara automatis ketika Jupyter Notebook and di "Run-All" oleh dosen (misal merubah ke csv dan upload ke GitHub).\
1-	Tentukan ketua kelompok, silahkan mulai fikirkan strategi pembagian kerja kelompoknya.\
2-	Semua menyiapkan Akun GitHub dan aplikasi GitHub Desktop (silahkan latihan menggunakan GitHub).\
3-	Pelajari dengan baik semua materi sebelumnya, latihan dengan data yang ada atau yang baru\
4-	Gunakan akumulasi semua pengetahuan/skill yang telah dipelajari sebelumnya untuk menjawab kasus saat ini.\
5-	Saat membuat laporan pengolahan data case study nanti, bayangkan kalian sedang bekerja di sebuah perusahaan sebagai Data Analyst dan membuat laporan ke atasan (bukan dosen). Perhatikan dengan baik narasi/kalimat yang digunakan dalam mengambil kesimpulan dan membuat rekomendasi.\
6-	Yakinkan Code anda dapat di "Run-All" oleh dosen di Google colaboratory.\
7-	Yakinkan anda memberikan informasi/insight serta rekomendasi/strategi Non-Trivial sebanyak-banyaknya\
8-	Yakinkan ada kesimpulan akhir (penutup) di bagian paling bawah "ipynb" anda.

## 1. Preparation

### 1.1 Libraries and Utilities

In [1]:
#Data Manipulatiom
import pandas as pd
import numpy as np
import sklearn
import scipy

#Visualization
import seaborn as sns           
import matplotlib.pyplot as plt 
%matplotlib inline 
sns.set(color_codes=True)

#Modelling
from imblearn import under_sampling, over_sampling
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score 
from sklearn.metrics import roc_auc_score
np.random.seed(1234)
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE
from sklearn.model_selection import RepeatedStratifiedKFold

### 1.2 Data Loading

### Import Data

In [2]:
# Loading Data:
df = pd.read_csv(r"C:\Users\USER\SEMESTER 7\Data Mining\Study Case 02 - Customer Segmentation\Case-Study-02.csv", low_memory = False, encoding='utf8')  

In [3]:
df.head()

,ID,Jenis Kelamin,Umur,Pendapatan-per-bulan,skor-pengeluaran,Menikah,Kota,Promo,HariFav,FavFlavor
0,101,Wanita,49.0,13114754.0,52,1,Jakarta,1.0,Sabtu,2
1,102,Wanita,32.0,33606557.0,86,1,Depok,1.0,Minggu,6
2,103,Pria,51.0,15983606.0,41,1,Jakarta,2.0,Minggu,3
3,104,Pria,19.0,12704918.0,55,0,Jakarta,1.0,Rabu,7
4,105,Wanita,38.0,20081967.0,42,1,Jakarta,2.0,Senin,8


In [4]:
df.tail()

,ID,Jenis Kelamin,Umur,Pendapatan-per-bulan,skor-pengeluaran,Menikah,Kota,Promo,HariFav,FavFlavor
195,296,Pria,31.0,4098360.0,73,1,Depok,1.0,Senin,0
196,297,Pria,57.0,15983606.0,51,1,Jakarta,1.0,Minggu,5
197,298,Wanita,35.0,43032786.0,79,1,Depok,1.0,Senin,7
198,299,Wanita,23.0,1229508.0,94,1,Depok,0.0,Selasa,0
199,300,Pria,50.0,1639344.0,14,1,Tangerang,3.0,Minggu,6


### 1.3 Data Understanding

Keterangan variabel:
- ID = ID customer, ditentukan lewat incremental value berdasarkan urutan kapan customer mendaftar loyality program
- Jenis Kelamin = Pria, Wanita
- Umur = Umur (dalam tahun)
- Pendapatan per-bulan = Pendapatan Customer dalam sebulan (Rupiah)
- Skor pengeluaran = Skor yang diberikan oleh MixITup untuk mengukur seberapa besar pengeluaran customer di MixITUp, Nilainya antara 0-100
- Menikah = Status pernikahan customer (0: Belum Menikah, 1: Menikah)
- Kota = Kota dimana customer mendafatarkan loyality programnya
- Promo = Seberapa sering customer menggunakan promonya: 0 (tidak pernah), 1 (jarang), 2 (sedang), 3 (sering)
- HariFav = Hari paling sering pelanggan menggunakan kartu loyality programnya.
- FavFlavor = Rasa Ice Cream yang paling sering dipilih customer
	0. Vanilla
	1. Strawberry
	3. Cokelat
	4. BlueBerry
	5. BubbleGum/Cotton Candy
	6. Coconut
	7. Caramel Pecan
	8. Mix Flavors

In [5]:
list(df.columns)

['ID',
 'Jenis Kelamin',
 'Umur',
 'Pendapatan-per-bulan',
 'skor-pengeluaran',
 'Menikah',
 'Kota',
 'Promo',
 'HariFav',
 'FavFlavor']

In [6]:
df.dtypes

ID                        int64
Jenis Kelamin            object
Umur                    float64
Pendapatan-per-bulan    float64
skor-pengeluaran          int64
Menikah                  object
Kota                     object
Promo                   float64
HariFav                  object
FavFlavor                 int64
dtype: object

Pada variabel "Umur" terdapat salah satu data yang berbentuk "35'" sehingga menyebabkan variabel tersebut berbentuk kategorik. Setelah itu, kami atasi permasalahan tersebut dengan menghapus tanda petiknya (') dengan menggunakan bantuan Ms. Excel. 

### Pengelompokkan Kolom Berdasarkan Tipe Data

In [8]:
# melihat kolom dengan tipe data yang berbeda (number dan object)

df_number = []
df_int = []
df_float = []
df_object = []

df_number.append(df.select_dtypes('number').columns.tolist()) # menampilkan nama kolom yang berisi data dengan tipe number
df_int.append(df.select_dtypes('integer').columns.tolist()) # menampilkan nama kolom yang berisi data dengan tipe number
df_float.append(df.select_dtypes('float').columns.tolist()) # menampilkan nama kolom yang berisi data dengan tipe number
df_object.append(df.select_dtypes('object').columns.tolist()) # menampilkan nama kolom yang berisi data dengan tipe object

print('Kolom bertipe Number (Integer/Float):\n\n',df_number,'\n')
print('Kolom bertipe Integer               :\n',df_int,'\n')
print('Kolom bertipe Float                 :\n',df_float,'\n')
print('Kolom bertipe Object                :\n\n',df_object)

Kolom bertipe Number (Integer/Float):

 [['ID', 'Umur', 'Pendapatan-per-bulan', 'skor-pengeluaran', 'Promo', 'FavFlavor']] 

Kolom bertipe Integer               :
 [['ID', 'skor-pengeluaran', 'FavFlavor']] 

Kolom bertipe Float                 :
 [['Umur', 'Pendapatan-per-bulan', 'Promo']] 

Kolom bertipe Object                :

 [['Jenis Kelamin', 'Menikah', 'Kota', 'HariFav']]


### Convert Integer to Float

In [9]:
#Feature yang data typenya integer
df_int = df.select_dtypes(include='int').columns
print(df_int)

Index(['ID', 'skor-pengeluaran', 'FavFlavor'], dtype='object')


In [10]:
df[df_int] = df[df_int].astype(float)

In [11]:
#Feature yang data typenya float
df_float = df.select_dtypes(include='float').columns
print(df_float)

Index(['ID', 'Umur', 'Pendapatan-per-bulan', 'skor-pengeluaran', 'Promo',
       'FavFlavor'],
      dtype='object')


### Data Preprocessing

In [12]:
# Total number of rows and columns
print("Dimensi data awal :", df.shape)
# df.shape

Dimensi data awal : (200, 10)


### Statistika Deskriptif

In [13]:
### Summary Statistics ###
df.describe()

,ID,Umur,Pendapatan-per-bulan,skor-pengeluaran,Promo,FavFlavor
count,200.000000,200.000000,2.000000e+02,200.00000,194.000000,200.000000
mean,200.500000,37.355000,1.867213e+07,50.47000,1.685567,3.605000
std,57.879185,11.992417,1.076423e+07,26.07101,0.970603,2.743466
min,101.000000,2.000000,0.000000e+00,1.00000,0.000000,0.000000
25%,150.750000,28.750000,1.086066e+07,34.75000,1.000000,1.000000
50%,200.500000,36.000000,1.905738e+07,50.00000,1.000000,3.500000
75%,250.250000,48.000000,2.581967e+07,73.00000,3.000000,6.000000
max,300.000000,60.000000,5.000000e+07,101.00000,5.000000,9.000000


### Handling Missing Value

It is used to remove outliers and duplicates from the dataset. Moreover, it is a very common practice to scale the columns on a standard scale, it helps in faster convergence and gives better results.

In [14]:
sum(df.duplicated())

0

In [15]:
#Remove Duplicate
df.drop_duplicates(inplace=True)
print(df.duplicated().sum()) # no more duplicates
print(df.shape) # re-check by printing data size

0
(200, 10)


tidak terdapat data duplikat di dalam data MixITUp

### Memilih data object

In [16]:
#Feature yang data typenya object
df_object = df.select_dtypes(include='object').columns
print(df_object)

Index(['Jenis Kelamin', 'Menikah', 'Kota', 'HariFav'], dtype='object')


In [17]:
obj = list(df_object)
df[obj].nunique()

Jenis Kelamin    3
Menikah          4
Kota             3
HariFav          7
dtype: int64

### Memilih data numerik

In [18]:
#Setelah menghapus beberapa fitur numerik
df_float = df.select_dtypes(include='float').columns
print(df_float)

Index(['ID', 'Umur', 'Pendapatan-per-bulan', 'skor-pengeluaran', 'Promo',
       'FavFlavor'],
      dtype='object')


### Drop Variabel yang Tidak Berguna

In [19]:
del df['ID']

variabel ID yang mana merupakan ID customer (ditentukan lewat incremental value berdasarkan urutan kapan customer mendaftar loyality program) tidak berguna sehingga variabel ID akan dihapus

In [20]:
#Setelah menghapus variabel
df_float = df.select_dtypes(include='float').columns
print(df_float)

Index(['Umur', 'Pendapatan-per-bulan', 'skor-pengeluaran', 'Promo',
       'FavFlavor'],
      dtype='object')


### Cek Missing Value

In [21]:
# Informasi Jumlah dan Persentase Missing Value

## general

df_mv = df.isnull().sum()                      # menampilkan jumlah total missing value per kolom
df_pmv = (df.isnull().sum()/len(df))*100     # menampilkan persentase missing value per kolom

df_missing = pd.concat([df_mv,df_pmv],axis = 1).reset_index() # menampilkan data dalam bentuk data frame

df_missing.columns = ['Kolom','Jumlah Missing Value','Persentase Missing Value']
missing_value = df_missing

print('Data Missing Value:\n')
print(missing_value.sample(5),'\n')
print('Persentase Missing Value Terbesar                  : ',missing_value['Persentase Missing Value'].max())
print('Persentase Missing Value Keseluruhan               : ',missing_value['Persentase Missing Value'].sum())


print('Data Duplikat:\n')
print('Jumlah terdapat data duplikat pada data df         : ',df.duplicated().sum(), '(',(df.duplicated().sum()/len(df))*100,'%',')')

Data Missing Value:

                  Kolom  Jumlah Missing Value  Persentase Missing Value
8             FavFlavor                     0                       0.0
2  Pendapatan-per-bulan                     0                       0.0
1                  Umur                     0                       0.0
0         Jenis Kelamin                     4                       2.0
7               HariFav                     0                       0.0 

Persentase Missing Value Terbesar                  :  6.0
Persentase Missing Value Keseluruhan               :  11.0
Data Duplikat:

Jumlah terdapat data duplikat pada data df         :  0 ( 0.0 % )


Pada data MixITUp tidak terdapat data yang duplikat sehingga tidak perlu dilakukan penghapusan data duplikat

In [22]:
print('Jumlah Data Missing Value:\n')
print(df.isnull().sum())

Jumlah Data Missing Value:

Jenis Kelamin            4
Umur                     0
Pendapatan-per-bulan     0
skor-pengeluaran         0
Menikah                 12
Kota                     0
Promo                    6
HariFav                  0
FavFlavor                0
dtype: int64


In [23]:
df.Menikah.value_counts()

1    160
0     26
       1
-      1
Name: Menikah, dtype: int64

In [26]:
df.dropna(subset=["Jenis Kelamin"], axis=0, inplace=True)
df.dropna(subset=["Promo"], axis=0, inplace=True)

akan dilakukan handling missing value terhadap data Jenis Kelamin dengan melakukan drop data yang missing pada variabel Jenis Kelamin karena total data missingnya ada 4 dari 200 atau 2% sehingga tidak terlalu mempengaruhi secara signifikan terhadap data keseluruhan. Selain itu, akan dilakukan handling missing value terhadap data Promo dengan melakukan drop data yang missing pada variabel Promo karena total data missingnya ada 6 dari 200 atau 3% sehingga tidak terlalu mempengaruhi secara signifikan terhadap data keseluruhan. Untuk missing value pada variabel Menikah tidak kami lakukan handling missing value karena variabel tersebut tidak kami gunakan dalam melakukan analisis

In [27]:
print('Jumlah Data Missing Value:\n')
print(df.isnull().sum())

Jumlah Data Missing Value:

Jenis Kelamin            0
Umur                     0
Pendapatan-per-bulan     0
skor-pengeluaran         0
Menikah                 12
Kota                     0
Promo                    0
HariFav                  0
FavFlavor                0
dtype: int64


## Hasil data setelah di cleansing

In [28]:
df.head()

,Jenis Kelamin,Umur,Pendapatan-per-bulan,skor-pengeluaran,Menikah,Kota,Promo,HariFav,FavFlavor
0,Wanita,49.0,13114754.0,52.0,1,Jakarta,1.0,Sabtu,2.0
1,Wanita,32.0,33606557.0,86.0,1,Depok,1.0,Minggu,6.0
2,Pria,51.0,15983606.0,41.0,1,Jakarta,2.0,Minggu,3.0
3,Pria,19.0,12704918.0,55.0,0,Jakarta,1.0,Rabu,7.0
4,Wanita,38.0,20081967.0,42.0,1,Jakarta,2.0,Senin,8.0


### Export Hasil Data Cleansing to csv file

In [30]:
df.to_csv("C://Users/USER/SEMESTER 7/Data Mining/Study Case 02 - Customer Segmentation/Data MixITUp Hasil Cleansing.csv")

Saran perbaikan data:
Karena masih terdapat missing value, noise berupa kesalahan entry nama pada variabel "Umur". Maka langkah yang perlu dilakukan adalah untuk missing value pada data akan dihilangkan pada data, yaitu akan dilakukan handling missing value terhadap data "Jenis Kelamin" dengan melakukan drop data yang missing pada variabel "Jenis Kelamin" karena total data missingnya ada 4 dari 200 atau 2% sehingga tidak terlalu mempengaruhi secara signifikan terhadap data keseluruhan. Selain itu, akan dilakukan handling missing value terhadap data "Promo" dengan melakukan drop data yang missing pada variabel "Promo" karena total data missingnya ada 6 dari 200 atau 3% sehingga tidak terlalu mempengaruhi secara signifikan terhadap data keseluruhan. Untuk missing value pada variabel "Menikah" tidak kami lakukan handling missing value karena variabel tersebut tidak kami gunakan dalam melakukan analisis. \\.

# Terima kasih